In [19]:
# Importing packages

import pandas as pd
import numpy as np

In [20]:
# Importing data
train_data = pd.read_csv('train_values.csv')
train_labels = pd.read_csv('train_labels.csv')
test_data = pd.read_csv('test_values.csv')

# combine train and test data
train_data['train'] = 1
test_data['train'] = 0
combined = pd.concat([train_data,test_data])

In [21]:
# Function to drop irrelevant columns, convert booleans, get top n values, one hot encode
# parameters are csv, number of top features to obtain, and flag for train vs test data
def preprocess(data,n):
    data = data.drop(columns = ['date_recorded','gps_height','longitude','latitude','wpt_name','num_private','subvillage','lga','ward','recorded_by','extraction_type_group','extraction_type','scheme_name','management','waterpoint_type_group','source','source_class','quantity_group','quality_group','payment_type'])
    
    data.permit = data.permit.astype(bool).astype(int)
    data.public_meeting = data.public_meeting.astype(bool).astype(int)
    
    def get_n(x,n):
    
        list_a = set(x.value_counts().index[:n].values) # Gets the top 50 values from the series
        x[~x.isin(list_a)] = 'Other' # vectorisation - boolean mask; tilda - inverting the boolean mask
        # faster than a for loop 
        return x

    data.funder = get_n(data.funder,n)
    data.installer = get_n(data.installer,n) 
    
    cat_columns = ['basin','scheme_management','quantity',
               'extraction_type_class', 'management_group','payment', 
               'water_quality', 'source_type','waterpoint_type',
               'region','installer', 'funder']

    data = pd.get_dummies(data,columns = cat_columns)

    return data

In [22]:
# process labels
label_dict_status_group = {'functional':0,
                   'non functional': 1,
                   'functional needs repair': 2}
train_labels.status_group = train_labels.status_group.replace(label_dict_status_group)

In [26]:
# create datasets for 50 top features
comb_50 = preprocess(combined,50)
comb_100 = preprocess(combined,100)

# split train test and drop additional column
train_50 = comb_50[comb_50['train']==1]
train_50 = train_50.drop(['train'],axis=1)
test_50 = comb_50[comb_50['train']==0]
test_50 = test_50.drop(['train'],axis=1)

# combining both train files
train_50 = pd.merge(train_50, train_labels, on = 'id')


# export as csv files
train_50.to_csv('train_50.csv', sep=',', encoding='utf-8', index=False)
test_50.to_csv('test_50.csv', sep=',', encoding='utf-8', index=False)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
